In [1]:
#Here are a set of libraries we imported to complete this assignment.
#Feel free to use these or equivalent libraries for your implementation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph
import os
import seaborn as sns # used for plot interactive graph.
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import sklearn.metrics.cluster as smc
from sklearn.model_selection import KFold


from matplotlib import pyplot
import itertools

%matplotlib inline

import random

random.seed(42)

In [2]:
pulse = pd.read_csv('pulse.csv')
eoc = pd.read_csv('filtered_EOC.csv')

In [4]:
pivoted_dfs = []

# Iterate over unique student IDs
for student_id in pulse['student_id'].unique():
    # Filter data for the current student ID
    student_data = pulse[pulse['student_id'] == student_id]
    # Pivot the DataFrame
    pivot_data = student_data.pivot_table(index=['student_id', 'chapter_number'], columns='construct', values='response').reset_index()
    # Append pivoted DataFrame to the list
    pivoted_dfs.append(pivot_data)

# Merge the pivoted DataFrames on 'student_id'
merged_df = pd.concat(pivoted_dfs).reset_index(drop=True)

In [5]:
merged_df.head()

construct,student_id,chapter_number,Cost,Expectancy,Intrinsic Value,Utility Value
0,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,2,4.0,4.0,4.0,4.0
1,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,3,4.0,3.0,4.0,4.0
2,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,5,3.0,2.0,3.0,3.0
3,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,6,3.0,3.0,3.0,3.0
4,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,8,2.0,2.0,2.0,2.0


In [6]:
merged_df['Pulse'] = merged_df[['Cost', 'Expectancy', 'Intrinsic Value', 'Utility Value']].mean(axis=1)

In [7]:
merged_df['chapter_number'] -= 1

In [23]:
eoc_pulse = pd.merge(merged_df, eoc, on=['student_id', 'chapter_number'], how='inner')

In [24]:
eoc_pulse.dropna(inplace=True)
eoc_pulse = eoc_pulse[['student_id', 'class_id', 'Pulse', 'EOC']]
eoc_pulse.head()

,student_id,class_id,Pulse,EOC
0,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,afcb6b4e-a0c0-46ce-b38c-c96329c91471,4.00,0.760000
1,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,afcb6b4e-a0c0-46ce-b38c-c96329c91471,3.75,0.514563
2,040852c0-3b9c-4fc6-8229-d7ba8264e9ce,afcb6b4e-a0c0-46ce-b38c-c96329c91471,3.00,0.436620
3,17c657d1-b46f-4210-8bce-0dddc75aa486,afcb6b4e-a0c0-46ce-b38c-c96329c91471,4.25,0.948718
4,17c657d1-b46f-4210-8bce-0dddc75aa486,afcb6b4e-a0c0-46ce-b38c-c96329c91471,4.50,0.914634


In [28]:
eoc_pulse = eoc_pulse.groupby(['student_id', 'class_id']).agg({'Pulse': 'mean', 'EOC': 'mean'}).reset_index()


In [29]:
eoc_pulse.head()

,student_id,class_id,Pulse,EOC
0,001824fb-a2fd-431d-aef6-7a1250d97a62,074123e7-cd90-4500-86fe-286aaa733bf5,4.159091,0.748087
1,0096491a-77bf-4e79-b906-fc7a8e5e57a6,b7489f46-1d28-4c69-9901-3fb8bf9b85a6,4.431818,0.567986
2,00fe5bb2-6b64-4a36-a706-eac4dc124ea7,51711479-441b-4c02-aef7-517aca63a53f,3.850000,0.849111
3,016e2fcf-0cc2-42ad-844d-e1057b5081cc,84444a0e-2905-46a8-bc93-5f566c0c1916,4.000000,0.854545
4,01805fac-7187-40e3-8a7e-1dda3c6c87f2,cc1ffb2e-5555-4109-8ad8-2d49cb54ad10,3.666667,0.638054


In [30]:
eoc_pulse.to_csv('eoc_pulse.csv', index=False)